# Classification:

While Linear Regression is used to predict a numeric value, classification is used to separate data into different labels. 

In this example, I will use the TensorFlow estimator to classify flowers.


Importing all the needed libraries:

In [ ]:
#importing TensorFlow and Pandas into this session.

import tensorflow as tf
import pandas as pd